In [1]:
import numpy as np
import tensorflow as tf
import xarray as xr
import time
from tensorflow.keras.models import load_model
%run "./topo_physics.ipynb"
%run "./utils.ipynb"
%run "./problems.ipynb"

In [2]:
def get_model(block_size, num_of_layers_1, num_of_layers_2, case_name):
  '''读取储存的训练好的模型'''
  cnn_model = load_model("Model/"+case_name+"/Models/cnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
  fnn_model = load_model("Model/"+case_name+"/Models/fnn_{layers_1}+{layers_2}layers_{Nb}xblock.h5".format(layers_1=num_of_layers_1,layers_2=num_of_layers_2, Nb=block_size))
  return cnn_model, fnn_model

def record_time_topo_simp(x=None, args=None):
  '''SIMP方法计时版'''
  # Root function that runs the full optimization routine
  if args is None:
    args = default_args()
  if x is None:
    x = np.ones((args['nely'], args['nelx'])) * args['volfrac']  # init mass
  total_start_time = time.time()
  frames = [x.copy()]
  ke = get_stiffness_matrix(args['young'], args['poisson'])  # stiffness matrix
  total_fem_cost = 0 
  losses, sens = [], []
  for step in range(args['opt_steps']):
    c, x, dc, fem_cost = record_time_optimality_criteria_step(x, ke, args)
    total_fem_cost += fem_cost
    losses.append(c)
    sens.append(dc)
    frames.append(x.copy())
  total_end_time = time.time()
  total_topo_cost = total_end_time - total_start_time
  return total_topo_cost, total_fem_cost 

def record_time_optimality_criteria_step(x, ke, args, penal=3, e_min=1e-9, e_0=1):
  fem_start_time = time.time()
  c, dc = autograd.value_and_grad(objective)(x, ke, args)
  fem_end_time = time.time()
  dv = autograd.grad(mean_density)(x, args)
  x = optimality_criteria_combine(x, dc, dv, args)
  fem_cost = fem_end_time - fem_start_time
  return c, x, dc, fem_cost


def run_topo_nn(args, problem, cnn_model, fnn_model, block_size, reduced_problem=None, verbose=False):
  '''嵌入神经网络的拓扑优化主函数（计时版）'''
  # Root function that runs the full optimization routine
  total_start_time = time.time()
  x = np.ones((args['nely'], args['nelx'])) * args['volfrac']  # init mass
  frames = [x.copy()]
  ke = get_stiffness_matrix(args['young'], args['poisson'])  # stiffness matrix
  if not reduced_problem:
    reduced_problem = scale_reduction_square(problem, factor=block_size)
  sens = []
  total_fem_time, total_nn_time = 0, 0
  for step in range(args['opt_steps']):
    x, dc, fem_time, nn_time = optimality_criteria_step_nn(x, args, reduced_problem, cnn_model, fnn_model, block_size)
    sens.append(dc)
    frames.append(x.copy())
    total_fem_time += fem_time
    total_nn_time += nn_time
  total_end_time = time.time()
  total_topo_time = total_end_time - total_start_time
  return frames, sens, total_topo_time, total_fem_time, total_nn_time

def optimality_criteria_step_nn(x, args, reduced_problem, cnn_model, fnn_model, block_size):
  fem_start_time = time.time()
  x_phys = physical_density(x, args, apply_cone_filter=True)
  # calculate reduced_u & reduced_sens and reshape them.
  x_phys_reduced = average_downsample(x_phys, factor=block_size)
  u, reduced_sens = fem_solver(x_phys_reduced, specified_task(reduced_problem))
  u = standardize(u)
  fem_end_time = time.time()
  ux, uy = separate_u(u, args['nelx']//block_size, args['nely']//block_size)
  ux_block = coarse_disp_to_block(ux)
  uy_block = coarse_disp_to_block(uy)
  disp_inputs = np.stack((ux_block, uy_block), axis=-1).reshape(-1,8)
  # calculate feature maps
  feature_maps = cnn_model(x_phys[None,:]).numpy() #(1,nely,nelx,n_filters)
  feature_blocks = features_to_block(feature_maps, block_size) #(num_of_blocks, block_size,block_size,n_filters)
  feature_inputs = feature_blocks.reshape(feature_blocks.shape[0],-1) #(num_of_blocks,16/36/64)
  fnn_inputs = [feature_inputs, disp_inputs]
  fnn_outputs = fnn_model(fnn_inputs).numpy() # (num_of_blocks, 4/9/16)
  sens_ratio = block_to_full(fnn_outputs, block_size, args['nely'], args['nelx']) #(nely, nelx)
  dc_phyx = ratio_to_value(sens_ratio, reduced_sens)
  dc = cone_filter(dc_phyx, args['filter_width'], args['mask'], transpose=True)
  nn_end_time = time.time()
  dv = autograd.grad(mean_density)(x, args)
  x = optimality_criteria_combine(x, dc, dv, args)
  fem_time = fem_end_time - fem_start_time
  nn_time = nn_end_time - fem_end_time
  return x, dc, fem_time, nn_time

def features_to_block(features, block_size):
  ''' (batch_size, nely, nelx, fillter_size)  into 
  (batch_size*(nely//block_size)*(nelx//blocksize), block_size, block_size)) blocks'''
  nely, nelx, n_filters = features.shape[1], features.shape[2], features.shape[3]
  num_of_slice = nelx//block_size
  transformed = np.moveaxis(tf.reshape(features,(-1,block_size,num_of_slice,block_size,n_filters)),1,2)
  return transformed.reshape(-1,block_size,block_size,n_filters)

from skimage.util import view_as_windows
def coarse_disp_to_block(image):
  '''rolling window with shape (2,2) and stride (1,1)'''
  return np.array(view_as_windows(image, window_shape=(2,2), step=(1,1))).reshape(-1,2,2)

def block_to_full(blocks, block_size, nely, nelx):
  '''(num_of_blocks, block_size**2) into (nely, nelx)'''
  num_of_blocks = blocks.shape[0]
  num_of_slice_x, num_of_slice_y = nelx//block_size, nely//block_size
  blocks = tf.reshape(blocks, (num_of_slice_y, num_of_slice_x, block_size, block_size))
  return tf.reshape(tf.experimental.numpy.moveaxis(blocks,1,2),(nely, nelx))

from skimage.transform import resize
def ratio_to_value(sens_ratio, sens_red):
  nely, nelx = sens_ratio.shape[0], sens_ratio.shape[1]
  sens_red_expanded = resize(sens_red, (nely,nelx), order=0, preserve_range=True)
  return sens_ratio * sens_red_expanded